In [1]:
#Import libraries for doing image analysis
from skimage.io import imread
from skimage.transform import resize
from sklearn.ensemble import RandomForestClassifier as RF
import glob
import os
from sklearn import cross_validation
from sklearn.cross_validation import StratifiedKFold as KFold
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
from matplotlib import colors
from pylab import cm
from skimage import segmentation
from skimage.morphology import watershed
from skimage import measure
from skimage import morphology
import numpy as np
import pandas as pd
from scipy import ndimage
from skimage.feature import peak_local_max
import tensorflow as tf
from math import ceil, floor
import time

# make graphics inline
%matplotlib inline

/home/davide/miniconda3/envs/honours/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import warnings
warnings.filterwarnings("ignore")

#### Importing the Data

In [3]:
# get the classnames from the directory structure
directory_names = list(set(glob.glob(os.path.join("competition_data","train_small", "*"))\
 ).difference(set(glob.glob(os.path.join("competition_data","train_small","*.*")))))

#### Preparing Training Data


In [4]:
# Rescale the images and create the combined metrics and training labels

#get the total training images
numberofImages = 0
for folder in directory_names:
    for fileNameDir in os.walk(folder):   
        for fileName in fileNameDir[2]:
             # Only read in the images
            if fileName[-4:] != ".jpg":
              continue
            numberofImages += 1
            
# We'll rescale the images to be 25x25
maxPixel = 28
imageSize = maxPixel * maxPixel
num_rows = numberofImages # one row for each image in the training dataset
num_features = imageSize # + 1 # for our ratio

# X is the feature vector with one row of features per image
# consisting of the pixel values and our metric
X = np.zeros((num_rows, num_features), dtype=float)
# y is the numeric class label 
y = np.zeros((num_rows))

files = []
# Generate training data
i = 0    
label = 0
# List of string of class names
namesClasses = list()

print("Reading images")
# Navigate through the list of directories
for folder in directory_names:
    # Append the string class name for each class
    currentClass = folder.split(os.pathsep)[-1]
    namesClasses.append(currentClass)
    for fileNameDir in os.walk(folder):   
        for fileName in fileNameDir[2]:
            # Only read in the images
            if fileName[-4:] != ".jpg":
              continue

            # Read in the images and create the features
            nameFileImage = "{0}{1}{2}".format(fileNameDir[0], os.sep, fileName)            
            image = imread(nameFileImage, as_grey=True)
            files.append(nameFileImage)
            #axisratio = getMinorMajorRatio(image)
            image = resize(image, (maxPixel, maxPixel))

            # Store the rescaled image pixels and the axis ratio
            X[i, 0:imageSize] = np.reshape(image, (1, imageSize))
            #X[i, imageSize] = axisratio

            # Store the classlabel
            y[i] = label
            i += 1
            # report progress for each 5% done  
            report = [int((j+1)*num_rows/10.) for j in range(10)]
            if i in report: 
                print (np.ceil(i *100.0 / num_rows), "% done")
    label += 1

Reading images
10.0 % done
20.0 % done
30.0 % done
40.0 % done
50.0 % done
60.0 % done
70.0 % done
80.0 % done
90.0 % done
100.0 % done


## Augmenting Dataset

In [5]:
IMAGE_SIZE = maxPixel

In [6]:
def flip_images(X_imgs):
    X_flip = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (IMAGE_SIZE, IMAGE_SIZE, 1))
    tf_img1 = tf.image.flip_left_right(X)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for img in X_imgs:
            flipped_imgs = sess.run([tf_img1], feed_dict = {X: img})
            X_flip.extend(flipped_imgs)
    X_flip = np.array(X_flip, dtype = np.float32)
    return np.concatenate((X_imgs, X_flip))

In [7]:
def rotate_images(X_imgs):
    X_rotate = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (IMAGE_SIZE, IMAGE_SIZE, 1))
    k = tf.placeholder(tf.int32)
    tf_img = tf.image.rot90(X, k = k)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for img in X_imgs:
            for i in range(3):  # Rotation at 90, 180 and 270 degrees
                rotated_img = sess.run(tf_img, feed_dict = {X: img, k: i + 1})
                X_rotate.append(rotated_img)
        
    X_rotate = np.array(X_rotate, dtype = np.float32)
    return np.concatenate((X_imgs, X_rotate))

In [8]:
def scale_images(X_imgs, scales=[0.80, 0.90]):
    # Various settings needed for Tensorflow operation
    boxes = np.zeros((len(scales), 4), dtype = np.float32)
    for index, scale in enumerate(scales):
        x1 = y1 = 0.5 - 0.5 * scale # To scale centrally
        x2 = y2 = 0.5 + 0.5 * scale
        boxes[index] = np.array([y1, x1, y2, x2], dtype = np.float32)
    box_ind = np.zeros((len(scales)), dtype = np.int32)
    crop_size = np.array([maxPixel, maxPixel], dtype = np.int32)
    
    X_scale_data = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (1, maxPixel, maxPixel, 1))
    # Define Tensorflow operation for all scales but only one base image at a time
    tf_img = tf.image.crop_and_resize(X, boxes, box_ind, crop_size)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for img_data in X_imgs:
            batch_img = np.expand_dims(img_data, axis = 0)
            scaled_imgs = sess.run(tf_img, feed_dict = {X: batch_img})
            X_scale_data.extend(scaled_imgs)
    
    X_scale_data = np.array(X_scale_data, dtype = np.float32)
    return np.concatenate((X_imgs, X_scale_data))

In [9]:
def get_translate_parameters(index):
    if index == 0: # Translate left 20 percent
        offset = np.array([0.0, 0.2], dtype = np.float32)
        size = np.array([IMAGE_SIZE, ceil(0.8 * IMAGE_SIZE)], dtype = np.int32)
        w_start = 0
        w_end = int(ceil(0.8 * IMAGE_SIZE))
        h_start = 0
        h_end = IMAGE_SIZE
    elif index == 1: # Translate right 20 percent
        offset = np.array([0.0, -0.2], dtype = np.float32)
        size = np.array([IMAGE_SIZE, ceil(0.8 * IMAGE_SIZE)], dtype = np.int32)
        w_start = int(floor((1 - 0.8) * IMAGE_SIZE))
        w_end = IMAGE_SIZE
        h_start = 0
        h_end = IMAGE_SIZE
    elif index == 2: # Translate top 20 percent
        offset = np.array([0.2, 0.0], dtype = np.float32)
        size = np.array([ceil(0.8 * IMAGE_SIZE), IMAGE_SIZE], dtype = np.int32)
        w_start = 0
        w_end = IMAGE_SIZE
        h_start = 0
        h_end = int(ceil(0.8 * IMAGE_SIZE)) 
    else: # Translate bottom 20 percent
        offset = np.array([-0.2, 0.0], dtype = np.float32)
        size = np.array([ceil(0.8 * IMAGE_SIZE), IMAGE_SIZE], dtype = np.int32)
        w_start = 0
        w_end = IMAGE_SIZE
        h_start = int(floor((1 - 0.8) * IMAGE_SIZE))
        h_end = IMAGE_SIZE 
        
    return offset, size, w_start, w_end, h_start, h_end

def translate_images(X_imgs):
    offsets = np.zeros((len(X_imgs), 2), dtype = np.float32)
    n_translations = 4
    X_translated_arr = []
    
    tf.reset_default_graph()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(n_translations):
            X_translated = np.zeros((len(X_imgs), IMAGE_SIZE, IMAGE_SIZE, 1), 
				    dtype = np.float32)
            X_translated.fill(1.0) # Filling background color
            base_offset, size, w_start, w_end, h_start, h_end = get_translate_parameters(i)
            offsets[:, :] = base_offset 
            glimpses = tf.image.extract_glimpse(X_imgs, size, offsets)
            
            glimpses = sess.run(glimpses)
            X_translated[:, h_start: h_start + size[0], \
			 w_start: w_start + size[1], :] = glimpses
            X_translated_arr.extend(X_translated)
    X_translated_arr = np.array(X_translated_arr, dtype = np.float32)
    return np.concatenate((X_imgs, X_translated_arr))

In [10]:
def augment_dataset(X_imgs, y_imgs):
    assert len(X_imgs) == len(y_imgs)
    X_aug = []
    y_aug = []
    i = 0
    print("Starting Dataset Augmentation...")
    
    for i in range(len(y_imgs)):
        
        imgs = [np.reshape(X_imgs[i,:],(IMAGE_SIZE,IMAGE_SIZE,1))]
        label = [y_imgs[i]]
        
        imgs = flip_images(imgs)
        imgs = rotate_images(imgs)
        imgs = scale_images(imgs)
        imgs = translate_images(imgs)
        
        labels = label * len(imgs)
#         print(len(labels), imgs.shape)
        
        X_aug += [np.array(x) for x in imgs.tolist()[:]]
        y_aug += labels
        
        i += 1
        # report progress for each 5% done  
        report = [int((j+1)*len(y_imgs)/10.) for j in range(10)]
        if i in report: 
            print (np.ceil(i *100.0 / len(y_imgs)), "% done")
        
    return X_aug, y_aug

## CNN for image classification

In [11]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [12]:
N_CLASSES = len(set(y))
print(N_CLASSES)

10


In [13]:
def cnn_model_fn(features, labels, mode):
    
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    # Plankton images are 25x25 pixels, and have one color channel
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    # Computes 32 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 28, 28, 1]
    # Output Tensor Shape: [batch_size, 28, 28, 32]
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 28, 28, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2
    # Computes 64 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 14, 14, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 64]
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 14, 14, 64]
    # Output Tensor Shape: [batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 7, 7, 64]
    # Output Tensor Shape: [batch_size, 7 * 7 * 64]
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 7 * 7 * 64]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=N_CLASSES)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [14]:
#Generate indexes to shuffle dataset and create testset

idx = np.arange(len(y))
np.random.seed(seed=1234)
np.random.shuffle(idx)
separator = ceil(len(y)/10*8)
print("Train / Total: ", separator, "/", len(y))

train_data = X[idx[:separator]].astype('float32')
eval_data = X[idx[separator:]].astype('float32')
train_labels = y[idx[:separator]].astype(int)
eval_labels = y[idx[separator:]].astype(int)

train_data = train_data #[:, 0:-1] # Returns np.array, remove axis rateo feature
eval_data = eval_data #[:, 0:-1]

print("Shapes:", train_data.shape, train_labels.shape, eval_data.shape, eval_labels.shape)

Train / Total:  2373 / 2966
Shapes: (2373, 784) (2373,) (593, 784) (593,)


In [15]:
def run_experiment(train_data, train_labels, eval_data, eval_labels, n_steps=1000):
    # Create the Estimator
#     dirname ="/tmp/models/convnet_model_" + str(time.time())[-5:]
#     print(dirname)
#     input()
    image_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/models/convnet_model_"+str(time.time()))

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
    image_classifier.train(
      input_fn=train_input_fn,
      steps=n_steps)#,
      #hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
    eval_results = image_classifier.evaluate(input_fn=eval_input_fn)
#     print(eval_results)
    return eval_results

In [16]:
basic_results = run_experiment(train_data, train_labels, eval_data, eval_labels)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/models/convnet_model_1519257822.4841151', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f74f0277550>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/models/convnet_model_1519257822.4841151/model.ckpt.
INFO:tensorflow:loss = 2.25693, step = 1
INFO:tensorflow:global_step/sec: 51.0752
INFO:tensorflow:loss = 1.84, step = 101 (1.959 sec)
INFO:tensorflow:global_step/sec: 49.5991
INFO:tensorflow:loss = 1.77395, step = 201 (2.016 sec)
INFO:tensorflow:

In [17]:
train_data_AUGM, train_labels_AUGM = augment_dataset(train_data, train_labels)

train_data_AUGM = np.array(train_data_AUGM, dtype='float32')
train_labels_AUGM = np.array(train_labels_AUGM, dtype='int')

print("Augmented dataset: ", train_data_AUGM.shape)

Starting Dataset Augmentation...
10.0 % done
20.0 % done
30.0 % done
40.0 % done
50.0 % done
60.0 % done
70.0 % done
80.0 % done
90.0 % done
100.0 % done
Augmented dataset:  (284760, 28, 28, 1)


In [18]:
augmented_results = run_experiment(train_data_AUGM, train_labels_AUGM, eval_data, eval_labels, n_steps=120*1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/models/convnet_model_1519258312.597218', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f74ec40d208>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/models/convnet_model_1519258312.597218/model.ckpt.
INFO:tensorflow:loss = 2.31322, step = 1
INFO:tensorflow:global_step/sec: 51.4917
INFO:tensorflow:loss = 2.09705, step = 101 (1.943 sec)
INFO:tensorflow:global_step/sec: 52.7298
INFO:tensorflow:loss = 1.90801, step = 201 (1.897 sec)
INFO:tensorflow

INFO:tensorflow:global_step/sec: 52.5265
INFO:tensorflow:loss = 1.64404, step = 7801 (1.904 sec)
INFO:tensorflow:global_step/sec: 52.7065
INFO:tensorflow:loss = 1.36146, step = 7901 (1.897 sec)
INFO:tensorflow:global_step/sec: 52.7887
INFO:tensorflow:loss = 0.993684, step = 8001 (1.894 sec)
INFO:tensorflow:global_step/sec: 52.6897
INFO:tensorflow:loss = 1.06483, step = 8101 (1.898 sec)
INFO:tensorflow:global_step/sec: 52.7904
INFO:tensorflow:loss = 1.39196, step = 8201 (1.894 sec)
INFO:tensorflow:global_step/sec: 52.7813
INFO:tensorflow:loss = 1.41876, step = 8301 (1.895 sec)
INFO:tensorflow:global_step/sec: 52.6462
INFO:tensorflow:loss = 1.18415, step = 8401 (1.900 sec)
INFO:tensorflow:global_step/sec: 52.3263
INFO:tensorflow:loss = 0.972785, step = 8501 (1.911 sec)
INFO:tensorflow:global_step/sec: 52.7265
INFO:tensorflow:loss = 1.36937, step = 8601 (1.897 sec)
INFO:tensorflow:global_step/sec: 52.6518
INFO:tensorflow:loss = 1.52694, step = 8701 (1.899 sec)
INFO:tensorflow:global_step/

INFO:tensorflow:global_step/sec: 52.0411
INFO:tensorflow:loss = 1.02242, step = 16201 (1.922 sec)
INFO:tensorflow:global_step/sec: 52.638
INFO:tensorflow:loss = 0.803809, step = 16301 (1.900 sec)
INFO:tensorflow:global_step/sec: 52.5824
INFO:tensorflow:loss = 1.47353, step = 16401 (1.902 sec)
INFO:tensorflow:global_step/sec: 52.7237
INFO:tensorflow:loss = 1.46475, step = 16501 (1.897 sec)
INFO:tensorflow:global_step/sec: 52.6457
INFO:tensorflow:loss = 0.774642, step = 16601 (1.900 sec)
INFO:tensorflow:global_step/sec: 52.718
INFO:tensorflow:loss = 0.990607, step = 16701 (1.897 sec)
INFO:tensorflow:global_step/sec: 52.8366
INFO:tensorflow:loss = 1.5396, step = 16801 (1.893 sec)
INFO:tensorflow:global_step/sec: 52.8382
INFO:tensorflow:loss = 1.05261, step = 16901 (1.892 sec)
INFO:tensorflow:global_step/sec: 52.655
INFO:tensorflow:loss = 1.24033, step = 17001 (1.899 sec)
INFO:tensorflow:global_step/sec: 52.6283
INFO:tensorflow:loss = 1.17857, step = 17101 (1.900 sec)
INFO:tensorflow:globa

INFO:tensorflow:global_step/sec: 52.5935
INFO:tensorflow:loss = 1.495, step = 24601 (1.901 sec)
INFO:tensorflow:global_step/sec: 52.7931
INFO:tensorflow:loss = 0.493965, step = 24701 (1.894 sec)
INFO:tensorflow:global_step/sec: 52.7022
INFO:tensorflow:loss = 0.570491, step = 24801 (1.897 sec)
INFO:tensorflow:global_step/sec: 52.6259
INFO:tensorflow:loss = 1.31596, step = 24901 (1.900 sec)
INFO:tensorflow:global_step/sec: 52.5352
INFO:tensorflow:loss = 1.56302, step = 25001 (1.904 sec)
INFO:tensorflow:global_step/sec: 52.6069
INFO:tensorflow:loss = 0.766725, step = 25101 (1.901 sec)
INFO:tensorflow:global_step/sec: 52.7122
INFO:tensorflow:loss = 1.17748, step = 25201 (1.898 sec)
INFO:tensorflow:global_step/sec: 52.5796
INFO:tensorflow:loss = 1.26541, step = 25301 (1.901 sec)
INFO:tensorflow:global_step/sec: 52.817
INFO:tensorflow:loss = 1.21844, step = 25401 (1.894 sec)
INFO:tensorflow:global_step/sec: 52.5443
INFO:tensorflow:loss = 0.930832, step = 25501 (1.903 sec)
INFO:tensorflow:glo

INFO:tensorflow:loss = 1.10956, step = 32801 (1.897 sec)
INFO:tensorflow:global_step/sec: 52.5006
INFO:tensorflow:loss = 0.895606, step = 32901 (1.905 sec)
INFO:tensorflow:global_step/sec: 52.6051
INFO:tensorflow:loss = 1.27962, step = 33001 (1.901 sec)
INFO:tensorflow:global_step/sec: 52.5426
INFO:tensorflow:loss = 1.12608, step = 33101 (1.903 sec)
INFO:tensorflow:global_step/sec: 52.6861
INFO:tensorflow:loss = 1.02636, step = 33201 (1.898 sec)
INFO:tensorflow:global_step/sec: 52.6094
INFO:tensorflow:loss = 3.19247, step = 33301 (1.901 sec)
INFO:tensorflow:global_step/sec: 52.8069
INFO:tensorflow:loss = 1.09286, step = 33401 (1.893 sec)
INFO:tensorflow:global_step/sec: 52.7149
INFO:tensorflow:loss = 0.818232, step = 33501 (1.897 sec)
INFO:tensorflow:global_step/sec: 52.772
INFO:tensorflow:loss = 1.06481, step = 33601 (1.895 sec)
INFO:tensorflow:global_step/sec: 52.7121
INFO:tensorflow:loss = 1.52912, step = 33701 (1.898 sec)
INFO:tensorflow:global_step/sec: 52.5997
INFO:tensorflow:los

...

INFO:tensorflow:global_step/sec: 50.7871
INFO:tensorflow:loss = 0.662039, step = 82801 (1.969 sec)
INFO:tensorflow:global_step/sec: 50.8935
INFO:tensorflow:loss = 0.702767, step = 82901 (1.965 sec)
INFO:tensorflow:global_step/sec: 50.5622
INFO:tensorflow:loss = 0.717137, step = 83001 (1.978 sec)
INFO:tensorflow:global_step/sec: 50.5719
INFO:tensorflow:loss = 0.466845, step = 83101 (1.977 sec)
INFO:tensorflow:global_step/sec: 50.6725
INFO:tensorflow:loss = 0.426001, step = 83201 (1.973 sec)
INFO:tensorflow:global_step/sec: 50.6968
INFO:tensorflow:loss = 0.78415, step = 83301 (1.973 sec)
INFO:tensorflow:global_step/sec: 50.5465
INFO:tensorflow:loss = 0.897075, step = 83401 (1.978 sec)
INFO:tensorflow:global_step/sec: 50.623
INFO:tensorflow:loss = 0.677132, step = 83501 (1.975 sec)
INFO:tensorflow:global_step/sec: 50.7355
INFO:tensorflow:loss = 0.321973, step = 83601 (1.971 sec)
INFO:tensorflow:global_step/sec: 50.699
INFO:tensorflow:loss = 0.441356, step = 83701 (1.972 sec)
INFO:tensorfl

INFO:tensorflow:loss = 0.65571, step = 91101 (1.975 sec)
INFO:tensorflow:global_step/sec: 50.6906
INFO:tensorflow:loss = 0.626409, step = 91201 (1.972 sec)
INFO:tensorflow:global_step/sec: 50.7475
INFO:tensorflow:loss = 0.827694, step = 91301 (1.971 sec)
INFO:tensorflow:global_step/sec: 50.8365
INFO:tensorflow:loss = 0.698084, step = 91401 (1.967 sec)
INFO:tensorflow:global_step/sec: 50.7654
INFO:tensorflow:loss = 0.49009, step = 91501 (1.970 sec)
INFO:tensorflow:global_step/sec: 50.9177
INFO:tensorflow:loss = 0.726464, step = 91601 (1.966 sec)
INFO:tensorflow:global_step/sec: 50.5666
INFO:tensorflow:loss = 0.753282, step = 91701 (1.975 sec)
INFO:tensorflow:global_step/sec: 50.7049
INFO:tensorflow:loss = 0.609023, step = 91801 (1.972 sec)
INFO:tensorflow:global_step/sec: 50.6081
INFO:tensorflow:loss = 0.993165, step = 91901 (1.976 sec)
INFO:tensorflow:global_step/sec: 50.7721
INFO:tensorflow:loss = 0.435961, step = 92001 (1.971 sec)
INFO:tensorflow:global_step/sec: 50.5824
INFO:tensorf

INFO:tensorflow:loss = 0.409869, step = 99301 (1.977 sec)
INFO:tensorflow:global_step/sec: 50.497
INFO:tensorflow:loss = 0.934458, step = 99401 (1.981 sec)
INFO:tensorflow:global_step/sec: 50.6981
INFO:tensorflow:loss = 0.845145, step = 99501 (1.972 sec)
INFO:tensorflow:global_step/sec: 50.7271
INFO:tensorflow:loss = 0.431593, step = 99601 (1.972 sec)
INFO:tensorflow:global_step/sec: 50.562
INFO:tensorflow:loss = 0.786532, step = 99701 (1.977 sec)
INFO:tensorflow:global_step/sec: 50.4297
INFO:tensorflow:loss = 1.20278, step = 99801 (1.983 sec)
INFO:tensorflow:global_step/sec: 50.6966
INFO:tensorflow:loss = 0.355803, step = 99901 (1.973 sec)
INFO:tensorflow:global_step/sec: 50.6029
INFO:tensorflow:loss = 0.322939, step = 100001 (1.977 sec)
INFO:tensorflow:global_step/sec: 50.7608
INFO:tensorflow:loss = 0.366308, step = 100101 (1.969 sec)
INFO:tensorflow:global_step/sec: 50.5897
INFO:tensorflow:loss = 0.471576, step = 100201 (1.977 sec)
INFO:tensorflow:global_step/sec: 50.6723
INFO:tenso

INFO:tensorflow:global_step/sec: 50.6782
INFO:tensorflow:loss = 0.779773, step = 107601 (1.973 sec)
INFO:tensorflow:global_step/sec: 50.52
INFO:tensorflow:loss = 0.205562, step = 107701 (1.979 sec)
INFO:tensorflow:global_step/sec: 50.5429
INFO:tensorflow:loss = 1.04794, step = 107801 (1.979 sec)
INFO:tensorflow:global_step/sec: 50.8351
INFO:tensorflow:loss = 0.613988, step = 107901 (1.967 sec)
INFO:tensorflow:global_step/sec: 50.6485
INFO:tensorflow:loss = 0.759666, step = 108001 (1.975 sec)
INFO:tensorflow:global_step/sec: 50.6539
INFO:tensorflow:loss = 0.426279, step = 108101 (1.974 sec)
INFO:tensorflow:global_step/sec: 50.8418
INFO:tensorflow:loss = 0.603371, step = 108201 (1.967 sec)
INFO:tensorflow:global_step/sec: 50.6899
INFO:tensorflow:loss = 0.486902, step = 108301 (1.973 sec)
INFO:tensorflow:global_step/sec: 50.7301
INFO:tensorflow:loss = 0.313913, step = 108401 (1.971 sec)
INFO:tensorflow:global_step/sec: 50.6897
INFO:tensorflow:loss = 0.657375, step = 108501 (1.974 sec)
INF

INFO:tensorflow:loss = 0.35275, step = 115801 (1.976 sec)
INFO:tensorflow:global_step/sec: 50.8457
INFO:tensorflow:loss = 0.57101, step = 115901 (1.967 sec)
INFO:tensorflow:global_step/sec: 51.1036
INFO:tensorflow:loss = 0.856816, step = 116001 (1.957 sec)
INFO:tensorflow:global_step/sec: 50.5624
INFO:tensorflow:loss = 0.452074, step = 116101 (1.978 sec)
INFO:tensorflow:global_step/sec: 50.7151
INFO:tensorflow:loss = 0.651196, step = 116201 (1.971 sec)
INFO:tensorflow:global_step/sec: 50.5915
INFO:tensorflow:loss = 0.773059, step = 116301 (1.977 sec)
INFO:tensorflow:global_step/sec: 50.84
INFO:tensorflow:loss = 0.428324, step = 116401 (1.967 sec)
INFO:tensorflow:global_step/sec: 50.7491
INFO:tensorflow:loss = 0.521663, step = 116501 (1.970 sec)
INFO:tensorflow:global_step/sec: 50.5994
INFO:tensorflow:loss = 0.482316, step = 116601 (1.978 sec)
INFO:tensorflow:global_step/sec: 50.6057
INFO:tensorflow:loss = 0.567301, step = 116701 (1.975 sec)
INFO:tensorflow:global_step/sec: 50.7209
INFO

In [19]:
print("BASIC RESULTS: ", basic_results)

BASIC RESULTS:  {'accuracy': 0.46205732, 'loss': 1.7478832, 'global_step': 1000}


In [20]:
print("AUGMENTED RESULTS: ", augmented_results)

AUGMENTED RESULTS:  {'accuracy': 0.68634063, 'loss': 0.91505849, 'global_step': 120000}


In [ ]:
"BASIC RESULTS:  {'accuracy': 0.67284989, 'loss': 1.0374305, 'global_step': 10000}"
"AUGMENTED RESULTS:  {'accuracy': 0.55311972, 'loss': 1.4051872, 'global_step': 10000}"

In [33]:
print("Training")
# n_estimators is the number of decision trees
# max_features also known as m_try is set to the default value of the square root of the number of features
clf = RF(n_estimators=100, n_jobs=3);
scores = cross_validation.cross_val_score(clf, X, y, cv=5, n_jobs=1);
print("Accuracy of all classes")
print(np.mean(scores))

Training
Accuracy of all classes
0.716075599594


In [34]:
kf = KFold(y, n_folds=5)
y_pred = y * 0
for train, test in kf:
    X_train, X_test, y_train, y_test = X[train,:], X[test,:], y[train], y[test]
    clf = RF(n_estimators=100, n_jobs=3)
    clf.fit(X_train, y_train)
    y_pred[test] = clf.predict(X_test)
print(classification_report(y, y_pred, target_names=namesClasses))

                                                               precision    recall  f1-score   support

                  competition_data/train_small/detritus_other       0.63      0.90      0.75       914
              competition_data/train_small/hydromedusae_typeF       0.80      0.61      0.69        61
competition_data/train_small/ctenophore_cydippid_no_tentacles       0.00      0.00      0.00        42
      competition_data/train_small/copepod_calanoid_eucalanus       0.97      0.38      0.54        96
            competition_data/train_small/detritus_filamentous       0.54      0.46      0.50       394
              competition_data/train_small/hydromedusae_typeE       0.00      0.00      0.00        14
               competition_data/train_small/ctenophore_cestid       1.00      0.08      0.15       113
                competition_data/train_small/crustacean_other       0.75      0.63      0.68       201
        competition_data/train_small/appendicularian_straight       0.69

The current model, while somewhat accurate overall, doesn't do well for all classes, including the shrimp caridean, stomatopod, or hydromedusae tentacles classes. For others it does quite well, getting many of the correct classifications for trichodesmium_puff and copepod_oithona_eggs classes. The metrics shown above for measuring model performance include precision, recall, and f1-score. The precision metric gives probability that a chosen class is correct, (true positives / (true positive + false positives)), while recall measures the ability of the model correctly classify examples of a given class, (true positives / (false negatives + true positives)). The F1 score is the geometric average of the precision and recall.

The competition scoring uses a multiclass log-loss metric to compute your overall score. In the next steps, we define the multiclass log-loss function and compute your estimated score on the training dataset.

In [35]:
def multiclass_log_loss(y_true, y_pred, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    https://www.kaggle.com/wiki/MultiClassLogLoss

    Parameters
    ----------
    y_true : array, shape = [n_samples]
            true class, intergers in [0, n_classes - 1)
    y_pred : array, shape = [n_samples, n_classes]

    Returns
    -------
    loss : float
    """
    predictions = np.clip(y_pred, eps, 1 - eps)

    # normalize row sums to 1
    predictions /= predictions.sum(axis=1)[:, np.newaxis]

    actual = np.zeros(y_pred.shape)
    n_samples = actual.shape[0]
    actual[np.arange(n_samples), y_true.astype(int)] = 1
    vectsum = np.sum(actual * np.log(predictions))
    loss = -1.0 / n_samples * vectsum
    return loss

In [36]:
# Get the probability predictions for computing the log-loss function
kf = KFold(y, n_folds=5)
# prediction probabilities number of samples, by number of classes
y_pred = np.zeros((len(y),len(set(y))))
for train, test in kf:
    X_train, X_test, y_train, y_test = X[train,:], X[test,:], y[train], y[test]
    clf = RF(n_estimators=100, n_jobs=3)
    clf.fit(X_train, y_train)
    y_pred[test] = clf.predict_proba(X_test)

In [37]:
multiclass_log_loss(y, y_pred)

0.96844833753168413

In [45]:
y.shape

(2966,)

## Evaluation with augmented dataset

In [48]:
print("Training")
# n_estimators is the number of decision trees
# max_features also known as m_try is set to the default value of the square root of the number of features
clf = RF(n_estimators=100, n_jobs=3);
scores = cross_validation.cross_val_score(clf, X_aug, y_aug, cv=5, n_jobs=1);
print("Accuracy of all classes")
print(np.mean(scores))

Training
Accuracy of all classes
0.742152463304


In [38]:
# Get the probability predictions for computing the log-loss function
kf = KFold(y_aug, n_folds=5)
# prediction probabilities number of samples, by number of classes
y_aug_pred = np.zeros((len(y_aug),len(set(y_aug))))
for train, test in kf:
    X_train, X_test, y_train, y_test = X_aug[train,:], X_aug[test,:], y_aug[train], y_aug[test]
    clf = RF(n_estimators=100, n_jobs=3)
    clf.fit(X_train, y_train)
    y_aug_pred[test] = clf.predict_proba(X_test)

In [40]:
multiclass_log_loss(y_aug, y_aug_pred)

0.83528301329128241

In [41]:
print(classification_report(y_aug, y_aug_pred, target_names=namesClasses))

ValueError: Mix type of y not allowed, got types {'continuous-multioutput', 'multiclass'}

In [47]:
y_aug_pred.shape

(142368, 10)

The multiclass log loss function is an classification error metric that heavily penalizes you for being both confident (either predicting very high or very low class probability) and wrong. Throughout the competition you will want to check that your model improvements are driving this loss metric lower.

#### Where to Go From Here

Now that you've made a simple metric, created a model, and examined the model's performance on the training data, the next step is to make improvements to your model to make it more competitive. The random forest model we created does not perform evenly across all classes and in some cases fails completely. By creating new features and looking at some of your distributions for the problem classes directly, you can identify features that specifically help separate those classes from the others. You can add new metrics by considering other image properties, stratified sampling, transformations, or other models for the classification.

Shapes: (2373, 626) (593, 626) (2373,) (593,)


In [43]:
len(y_test)

593

In [57]:
 len(set(train_labels))

10

In [77]:
type(train_data[0,0])

numpy.float32